# Decode VIN


This notebook uses the truncated VIN (Vehicle Identification Number) in the MV104 vehicle dataset and NHTSA's VIN API to get vehicle body type information. 

The MV104 data from the DMV has a variable VEHBDT_ID that referes to the vehicle body type, but it doesn't match what we see from the NHTSA's decoding of the VIN. 

In particular DMV has a vehicle type 'Suburban'. Not immediately clear what that referes too. Turns out, it seems it is any light duty vehicle that is not a car (eg SUV's and pickups).

NHTSA's vehicle type is more descriptive than the DMV variable.

In [21]:
import pandas as pd
pd.options.display.max_columns = 130
pd.options.display.max_rows = 130


import databuild as db
import sys
sys.path.insert(0,'/home/deena/Documents/data_munge/ModaCode/')
import moda


In [2]:
# read in DMV MV104 data. There are 3 tables. 
crash,ind,veh = db.readDMV()

# we're interested in the vehicle table which has VIN
print veh.shape

/usr/local/lib/python2.7/dist-packages/pandas/core/computation/check.py:17: UserWarning: The installed version of numexpr 2.4.3 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)


full crash table (522108, 26)
full person table (1502797, 22)
full vehicle table (1092922, 20)
(1092922, 20)


In [3]:
# Remove null vins
veh = veh[pd.notnull(veh['VIN'])].copy()

veh['CV_VEH_YEAR'] = veh['CV_VEH_YEAR'].astype(str)
veh['VIN'] = veh['VIN'].astype(str)

print veh.shape

(611697, 20)


In [11]:
veh.head()

,CS_ID,CV_ID,VEHBDYT_ID,REGT_CDE,PACCACTT_ID,TBCT_DMV_CDE,DIRCTT_CDE,FT_ID,CV_VEH_YEAR,CV_REG_STATE_CDE,CV_WEIGHT_LBS,CV_PSGR_NUM,CV_CYLNDR,CV_VEHMAKE_DESCR,CFT_CDE1,CFT_CDE2,SHZMTT_ID,VEH_EVNTT_ID,VIN,DMV_VIN_NUM
0,32045885,5979599,6.0,16,-1,-3,-1,1,1998,NY,None,-1,4,TOYOT,None,None,None,None,2T1BR12E2WC,None
2,32045886,5979601,5.0,16,1,-3,-1,1,1999,NY,None,2,6,DODGE,None,None,None,None,2B4FP2532XR,None
3,32045887,5979602,60.0,56,1,-3,-1,2,2002,NY,None,-1,8,GMC,None,None,None,None,1GDJG31F821,None
6,32046208,5980144,5.0,29,10,-3,-1,1,1996,NY,None,-1,4,SUBAR,None,None,None,None,4S3BK6657T7,None
7,32046209,5980145,5.0,16,1,-3,W,1,1997,NY,None,1,8,MERCU,-1,None,None,None,4M2DU55P6VU,None


In [5]:
# Take a random sample of vehicles
veh_sample = veh.sample(n=5000)

# create a dataframe to store the vehicle information
# obtained from the NHTSA vehicle api 
vin_api_data = pd.DataFrame()

nhtsaURL1 = 'https://vpic.nhtsa.dot.gov/api/vehicles/decodevinvalues/'
nhtsaURL2 = '*BA?format=json&modelyear='


for index, row in veh_sample.iterrows():
    vin = row['VIN']
    year = row['CV_VEH_YEAR']
    vin_data = pd.read_json(nhtsaURL1+vin+nhtsaURL2+year)
    vin_norm = pd.DataFrame(vin_data['Results'][0],index=[0])
    
    # Add each line of results (individual API call) to vin_api_data 
    vin_api_data = pd.concat([vin_api_data, vin_norm], axis=0)

print vin_api_data.shape

(5000, 146)


In [6]:
# join back to crash table
# Take first 11 digits of the API result's vin
vin_api_data['VIN'] = vin_api_data['VIN'].str[:-3]

# Use that 11 digit vin to join with the original vehicle table
veh_sample = veh_sample.merge(vin_api_data,how='left',on='VIN')
veh_sample.head()

,CS_ID,CV_ID,VEHBDYT_ID,REGT_CDE,PACCACTT_ID,TBCT_DMV_CDE,DIRCTT_CDE,FT_ID,CV_VEH_YEAR,CV_REG_STATE_CDE,...,Turbo,ValveTrainDesign,VehicleType,WheelBaseLong,WheelBaseShort,WheelBaseType,WheelSizeFront,WheelSizeRear,Wheels,Windows
0,34733611,11036207,5.0,16,2,-3,E,1,2001,NY,...,,,TRUCK,,,,,,,
1,33385538,8505537,6.0,16,14,-3,E,1,2006,NY,...,,,PASSENGER CAR,,,,,,,
2,32633044,7349171,6.0,16,11,-3,W,1,2001,NY,...,,Single Overhead Cam (SOHC),PASSENGER CAR,,,,,,,
3,32537313,6913053,5.0,16,1,-3,W,1,1999,NY,...,,Dual Overhead Cam (DOHC),MULTIPURPOSE PASSENGER VEHICLE (MPV),,,,,,,
4,34630872,10851608,5.0,16,7,-3,W,1,2000,NY,...,,,MULTIPURPOSE PASSENGER VEHICLE (MPV),,,,,,,


the VIN API has has two variables that refer to type of vehicle:
VehicleType and BodyClass

Here's how they relate to each other:

In [7]:
veh_sample.groupby(['VehicleType','BodyClass'])[['VIN']].count()

VIN
VehicleType                          BodyClass                                               
                                                                                           19
BUS                                                                                         1
                                     Bus                                                   38
                                     Bus - School Bus                                       2
                                     Incomplete - School Bus Chassis                        2
                                     Truck                                                  8
                                     Van                                                   15
                                     Wagon                                                 22
INCOMPLETE VEHICLE                                                                          1
                                     Cargo Van                                              2
                                     Incomplete                                             6
                                     Incomplete - Bus Chassis                               8
                                     Incomplete - Chassis Cab (Number of Cab Unknown)      19
                                     Incomplete - Chassis Cab (Single Cab)                 23
                                     Incomplete - Commercial Bus Chassis                    4
                                     Incomplete - Cutaway                                  42
                                     Incomplete - Motor Home Chassis                        1
                                     Incomplete - School Bus Chassis                        3
                                     Incomplete - Shuttle Bus Chassis                       1
                                     Incomplete - Stripped Chassis                          3
                                     Truck                                                 87
                                     Van                                                   17
                                     Wagon                                                  2
MOTORCYCLE                                                                                  2
                                     Motorcycle - Cruiser                                   1
                                     Motorcycle - Custom                                    8
                                     Motorcycle - Dual Sport / Adventure / Supermoto...     2
                                     Motorcycle - Scooter                                   7
                                     Motorcycle - Sport                                    33
                                     Motorcycle - Standard                                  2
                                     Motorcycle - Street                                    1
                                     Motorcycle - Touring / Sport Touring                   5
MULTIPURPOSE PASSENGER VEHICLE (MPV)                                                       23
                                     Crossover Utility Vehicle (CUV)                       67
                                     Hatchback/Liftback/Notchback                          14
                                     Incomplete - Chassis Cab (Number of Cab Unknown)       1
                                     Minivan                                              109
                                     Pickup                                                 8
                                     Sport Utility Vehicle (SUV)/Multi Purpose Vehic...   931
                                     Sport Utility Vehicle (SUV)/Multi Purpose Vehic...     5
                                     Van                                                  186
                                     Wagon      

The DMV vehicle file has a column called 'VEHBDYT_ID" which contains the DMV encoding of the body type. 

First let's make the variable more descriptive by replacing the numbers by their descriptive value from the data dictionary.

Then let's compare to the NHTSA body type.

In [17]:
veh_sample = db.formatVehTypeDMV(veh_sample)

In [18]:
# compare counts of DMV vehicle type with NHTSA's VIN decoding 'VehicleType'

veh_sample.groupby(['DMV_VehType','VehicleType'])[['VIN']].count()

VIN
DMV_VehType          VehicleType                               
2 Door Sedan                                                  1
                     MULTIPURPOSE PASSENGER VEHICLE (MPV)     2
                     PASSENGER CAR                          208
4 Door Sedan                                                  4
                     INCOMPLETE VEHICLE                       1
                     MULTIPURPOSE PASSENGER VEHICLE (MPV)    59
                     PASSENGER CAR                         2832
Ambulance            INCOMPLETE VEHICLE                      11
Bus (Omnibus)        BUS                                     53
                     INCOMPLETE VEHICLE                      22
                     TRUCK                                    1
Cement Mixer         INCOMPLETE VEHICLE                       1
Convertible          PASSENGER CAR                           26
Custom                                                        2
Delivery Truck       INCOMPLETE VEHICLE                      27
                     TRUCK                                    2
Dump                                                          1
                     INCOMPLETE VEHICLE                      47
                     TRUCK                                    2
Earth Mover          INCOMPLETE VEHICLE                       1
Flat Bed Truck       INCOMPLETE VEHICLE                       6
                     TRUCK                                    1
Motorcycle           MOTORCYCLE                              61
Pickup Truck                                                  1
                     MULTIPURPOSE PASSENGER VEHICLE (MPV)     1
                     TRUCK                                   71
Refrigerator Truck   INCOMPLETE VEHICLE                      11
                     TRUCK                                    2
Road Sweeper                                                  4
Sand Or Agricultural INCOMPLETE VEHICLE                       1
Sedan                PASSENGER CAR                            1
Stake Truck          INCOMPLETE VEHICLE                       2
                     TRUCK                                    1
Suburban                                                      2
                     BUS                                     29
                     INCOMPLETE VEHICLE                      38
                     MULTIPURPOSE PASSENGER VEHICLE (MPV)  1697
                     PASSENGER CAR                           68
                     TRUCK                                   50
Tank Truck           INCOMPLETE VEHICLE                       1
Taxi                 INCOMPLETE VEHICLE                       1
                     MULTIPURPOSE PASSENGER VEHICLE (MPV)    94
                     PASSENGER CAR                          401
                     TRUCK                                    2
Tow Truck            INCOMPLETE VEHICLE                       4
Tractor                                                       2
                     TRUCK                                   20
Truck Crane          TRUCK                                    1
Unknown Vehicle                                               1
Utility              INCOMPLETE VEHICLE                      12
                     MULTIPURPOSE PASSENGER VEHICLE (MPV)     1
                     TRUCK                                    1
Van Truck                                                     1
                     BUS                                      6
                     INCOMPLETE VEHICLE                      33
                     MULTIPURPOSE PASSENGER VEHICLE (MPV)    13
                     PASSENGER CAR                            1
                     TRUCK                                   89

In [22]:
# compare counts of DMV vehicle type with NHTSA's VIN decoding 'BodyClass'


veh_sample.groupby(['DMV_VehType','BodyClass'])[['VIN']].count()

VIN
DMV_VehType          BodyClass                                               
2 Door Sedan                                                                5
                     Cabriolet/Convertible                                  6
                     Coupe                                                137
                     Hatchback/Liftback/Notchback                          45
                     Sedan/Saloon                                          16
                     Sport Utility Vehicle (SUV)/Multi Purpose Vehic...     1
                     Van                                                    1
4 Door Sedan                                                               15
                     Coupe                                                  6
                     Hatchback/Liftback/Notchback                          53
                     Minivan                                                3
                     Sedan/Saloon                                        2757
                     Sedan/Saloon, Hatchback/Liftback/Notchback             1
                     Sport Utility Vehicle (SUV)/Multi Purpose Vehic...    26
                     Van                                                    1
                     Wagon                                                 34
Ambulance            Incomplete - Chassis Cab (Number of Cab Unknown)       5
                     Incomplete - Chassis Cab (Single Cab)                  6
Bus (Omnibus)                                                               1
                     Bus                                                   38
                     Bus - School Bus                                       2
                     Cargo Van                                              1
                     Incomplete - Bus Chassis                               8
                     Incomplete - Commercial Bus Chassis                    3
                     Incomplete - Cutaway                                   5
                     Incomplete - Motor Home Chassis                        1
                     Incomplete - School Bus Chassis                        5
                     Incomplete - Shuttle Bus Chassis                       1
                     Truck                                                  8
                     Van                                                    1
                     Wagon                                                  2
Cement Mixer         Truck                                                  1
Convertible          Cabriolet/Convertible                                 26
Custom                                                                      2
Delivery Truck       Incomplete                                             2
                     Incomplete - Chassis Cab (Number of Cab Unknown)       2
                     Incomplete - Chassis Cab (Single Cab)                  4
                     Incomplete - Cutaway                                   1
                     Truck                                                 18
                     Van                                                    2
Dump                                                                        1
                     Incomplete - Chassis Cab (Number of Cab Unknown)       3
                     Truck                                                 46
Earth Mover          Incomplete - Chassis Cab (Number of Cab Unknown)       1
Flat Bed Truck       Incomplete - Chassis Cab (Single Cab)                  3
                     Truck                                                  4
Motorcycle                                                                  2
                     Motorcycle - Cruiser                                   1
                     Motorcycle - Custom                                    8
                     Motorcycle - Dual Sport / Adventure / Supermoto...     2
                  

VEHBDYT_ID = 5 ("Suburban") matches primarily to SUV's, but almost as many are Vans and Wagons.
